In [3]:
#!pip install PyAutoGUI

In [7]:
import pyautogui

pyautogui.alert('This is an alert box.')

'OK'

In [8]:
pyautogui.confirm('Shall I proceed?')

'Cancel'

In [14]:
screenWidth, screenHeight = pyautogui.size() # Returns two integers, the width and height of the screen. (The primary monitor, in multi-monitor setups.)
print(screenWidth, screenHeight)

1920 1080


In [10]:
pyautogui.moveTo(100, 150)

In [ ]:
# quando ele roda
# se rodou com sucesso ou nao
# sem tem a mensageria
# tempo de execucao
# quais pipes tem os alertas (metas)


In [4]:
import os
import json
import re
# Path to the folder

folder_path = r'C:\Users\PC\Desktop\python\mapeamento_adf\pipeline\pipeline'

# List all files in the folder
files = os.listdir(folder_path)
# ['biz_biz_analitico_ep.json', 'biz_biz_base_unimercado_ep.json', 'biz_biz_base_unimercado_ssi.json', 'biz_biz_base_unimercado_sti.json', ...

result_list = [] 
for file_name in files:
    # biz_biz_analitico_ep.json
    # biz_biz_base_unimercado_ep.json
    # biz_biz_base_unimercado_ssi.json
    # biz_biz_base_unimercado_sti.json
    # biz_biz_calcula_coluna.json


    file_path = os.path.join(folder_path, file_name)
    # 'C:\Users\PC\Desktop\mapeamento_adf\pipeline\pipeline' + ['biz_biz_analitico_ep.json', 'biz_biz_base_unimercado_ep.json', 'biz_biz_base_unimercado_ssi.json', 'biz_biz_base_unimercado_sti.json', ...
    with open(file_path, 'r') as file:
        data = json.load(file)

        # Now 'data' contains the JSON data as a Python dictionary
        # Convert the JSON object to a text string
        json_string = json.dumps(data, indent=2)
        word = 'lnd'
        layer = 'lnd_org_raw'
        if word in json_string:
            # if re.search(rf'\b{word}\b', json_string, re.IGNORECASE) and not file_name.startswith('wkf') and file_name.startswith(layer):
            if word in json_string and file_name.startswith(layer) and not file_name.startswith('wkf'): 
                #>> it doesn't work, cause the expression 'if word in json_string' ain't looking for the 'word in the specific sequence'
                #print(file_name)
                with open(file_path, 'r') as file:
                    #data = json.load(file)
                    #print(data)
                    name_pipeline = data.get('name', {})
                    folder = data.get('properties', {}).get('folder', {}).get('name', {})
                    bot_script = data.get('properties', {}).get('parameters', {}).get('bot', {}).get('defaultValue', {})
                    #.get('parameters', {}).get('tables', {}).get('defaultValue', {}).get('path_destination')
                    #print(name_pipeline)
                    #print(folder)
                    #print(bot_script)

                    dados = {
                    'NAME_PIPELINE': name_pipeline,
                    'FOLDER': folder,
                    'BOT_SCRIPT_NAME': bot_script}

                    result_list.append(dados)

# With API

In [ ]:

    {
        "id": "/subscriptions/3e3a21e3-9551-4945-8c46-c02f558392ce/resourceGroups/bigdata/providers/Microsoft.DataFactory/factories/cnibigdatafactory/pipelines/lnd_org_raw_trello",
        "name": "lnd_org_raw_trello",
        "type": "Microsoft.DataFactory/factories/pipelines",
        "properties": {
            "activities": [
                {
                    "name": "import_lnd__0__execute_crw",
                    "type": "ExecutePipeline",
                    "state": "Inactive",
                    "onInactiveMarkAs": "Failed",
                    "dependsOn": [],
                    "policy": {
                        "secureInput": false
                    },
                    "userProperties": [],
                    "typeProperties": {
                        "pipeline": {
                            "referenceName": "import_lnd__0__execute_crw",
                            "type": "PipelineReference"
                        },
                        "waitOnCompletion": true,
                        "parameters": {
                            "bot": {
                                "value": "@pipeline().parameters.bot",
                                "type": "Expression"
                            },
                            "adf": {
                                "value": "@json(concat('{\"adf_factory_name\": \"', pipeline().DataFactory, '\", \"adf_pipeline_name\": \"', pipeline().Pipeline, '\", \"adf_pipeline_run_id\": \"', pipeline().RunId, '\",  \"adf_trigger_id\": \"', pipeline().TriggerId, '\", \"adf_trigger_name\": \"', pipeline().TriggerName, '\", \"adf_trigger_time\": \"', pipeline().TriggerTime, '\",  \"adf_trigger_type\": \"', pipeline().TriggerType, '\"}'))",
                                "type": "Expression"
                            },
                            "env": {
                                "value": "@pipeline().parameters.env",
                                "type": "Expression"
                            },
                            "bot_params": {
                                "value": "@pipeline().parameters.bot_params",
                                "type": "Expression"
                            }
                        }
                    }
                },
                {
                    "name": "alerta_falha",
                    "type": "ExecutePipeline",
                    "state": "Inactive",
                    "onInactiveMarkAs": "Succeeded",
                    "dependsOn": [
                        {
                            "activity": "import_lnd__0__execute_crw",
                            "dependencyConditions": [
                                "Failed"
                            ]
                        }
                    ],
                    "policy": {
                        "secureInput": false
                    },
                    "userProperties": [],
                    "typeProperties": {
                        "pipeline": {
                            "referenceName": "send_email",
                            "type": "PipelineReference"
                        },
                        "waitOnCompletion": true,
                        "parameters": {
                            "email_users": {
                                "value": "@array('monitoramento_observatorio@cni.com.br')",
                                "type": "Expression"
                            },
                            "email_groups": {
                                "value": "@array('')",
                                "type": "Expression"
                            },
                            "adf": {
                                "value": "@json(concat('{\"adf_factory_name\": \"', pipeline().DataFactory, '\", \"adf_pipeline_name\": \"', pipeline().Pipeline, '\", \"adf_pipeline_run_id\": \"', pipeline().RunId, '\",  \"adf_trigger_id\": \"', pipeline().TriggerId, '\", \"adf_trigger_name\": \"', pipeline().TriggerName, '\", \"adf_trigger_time\": \"', pipeline().TriggerTime, '\",  \"adf_trigger_type\": \"', pipeline().TriggerType, '\"}'))",
                                "type": "Expression"
                            },
                            "env": {
                                "value": "@pipeline().parameters.env",
                                "type": "Expression"
                            }
                        }
                    }
                }

In [ ]:
# Load JSON data
data = json.loads(json_data)

# Extract email_users from the nested structure
email_users = None
for activity in data['properties']['activities']:
    if activity['name'] == 'alerta_falha':
        email_users = activity['typeProperties']['parameters']['email_users']['value']
        break

print("Email Users:", email_users)

In [3]:
import os
import re
import json

import pandas as pd


# Path to the folder

#json_file = r'C:\Users\PC\Desktop\python\mapeamento_adf\all_pipelines.json'
json_file = '././all_pipelines.json'
with open(json_file, 'r') as file:
    json_string = json.load(file)
    word = 'lnd'
    layer = 'lnd_org_raw'
    #print(len(json_string))
    result_list = []
    for x in json_string:
        if 'name' in x and x['name'].startswith(layer) and not x['name'].startswith('wkf'):
            name_pipeline = x.get('name', {})
            print(name_pipeline)
            folder = x.get('properties', {}).get('folder', {}).get('name', {})
            bot_script = x.get('properties', {}).get('parameters', {}).get('bot', {}).get('defaultValue', {})
            email_users = 'Não existe'
            for z in x["properties"]['activities']:
                try:
                    if z['name'] == 'alerta_falha':
                        email_users = z['typeProperties']['parameters']['email_users']['value']
                        print(email_users)
                except:
                    pass

            dados = {
            'NAME_PIPELINE': name_pipeline,
            'FOLDER': folder,
            'BOT_SCRIPT_NAME': bot_script,
            'MONITORAMENTO': email_users}
            result_list.append(dados)
        else:
            pass



# Assuming result_list is your list of dictionaries
df = pd.DataFrame(result_list)
df

lnd_org_raw_censo_escolar
lnd_org_raw_microdados_enem
lnd_org_raw_enem_escola
lnd_org_raw_saeb
lnd_org_raw_estrutura_territorial
lnd_org_raw_estrutura_cnae
lnd_org_raw_microdados_pnad
lnd_org_raw_inse_nivel_socioeconomico
lnd_org_raw_ipca
lnd_org_raw_estrutura_cbo
lnd_org_raw_microdados_caged_ajustes
lnd_org_raw_rais_estabelecimento
lnd_org_raw_prova_brasil
lnd_org_raw_microdados_caged
lnd_org_raw_ideb
lnd_org_raw_inse_afd
lnd_org_raw_rais_vinculo
lnd_org_raw_trello
@array('monitoramento_observatorio@cni.com.br')
lnd_org_raw_deflatores
lnd_org_raw_scnt
lnd_org_raw_inep_atu
lnd_org_raw_inep_had
lnd_org_raw_inep_ird
lnd_org_raw_tdi
lnd_org_raw_dsu
lnd_org_raw_inep_rmd
lnd_org_raw_catho_vagas
lnd_org_raw_vagascom_vagas
lnd_org_raw_infojobs_vagas
lnd_org_raw_vagascertas_vagas
lnd_org_raw_ibge_pop_projetada
lnd_org_raw_ibge_pop_estimada
lnd_org_raw_ibge_pintec_tipo_programa
lnd_org_raw_ibge_pintec_org_mkt
lnd_org_raw_ibge_pintec_bio_nano
lnd_org_raw_inss_cat
lnd_org_raw_microdados_novo_cage

,NAME_PIPELINE,FOLDER,BOT_SCRIPT_NAME,MONITORAMENTO
0,lnd_org_raw_censo_escolar,lnd/crw/inep_censo_escolar,org_raw_censo_escolar,Não existe
1,lnd_org_raw_microdados_enem,lnd/crw/inep_enem,org_raw_microdados_enem,Não existe
2,lnd_org_raw_enem_escola,lnd/crw/inep_enem,org_raw_enem_escola,Não existe
3,lnd_org_raw_saeb,lnd/crw/inep_saeb,org_raw_saeb,Não existe
4,lnd_org_raw_estrutura_territorial,lnd/crw/ibge,org_raw_estrutura_territorial,Não existe
...,...,...,...,...
180,lnd_org_raw_mcti_empresas_lei_do_bem,lnd/crw/oni/mcti,org_raw_mcti_empresas_lei_do_bem,@array('monitoramento_observatorio@cni.com.br')
181,lnd_org_raw_bndes_operacoes_indiretas_automaticas,lnd/crw/oni/bndes,org_raw_bndes_operacoes_indiretas_automaticas,@array('monitoramento_observatorio@cni.com.br')
182,lnd_org_raw_cnpq_financiamento,lnd/crw/oni/cnpq,org_lnd_raw_cnpq_financiamento,@array('monitoramento_observatorio@cni.com.br')
183,lnd_org_raw_bndes_operacoes_nao_automaticas,lnd/crw/oni/bndes,org_raw_bndes_operacoes_nao_automaticas,@array('monitoramento_observatorio@cni.com.br')


,NAME_PIPELINE,FOLDER,BOT_SCRIPT_NAME,MONITORAMENTO
0,lnd_org_raw_censo_escolar,lnd/crw/inep_censo_escolar,org_raw_censo_escolar,Não existe
1,lnd_org_raw_microdados_enem,lnd/crw/inep_enem,org_raw_microdados_enem,Não existe
2,lnd_org_raw_enem_escola,lnd/crw/inep_enem,org_raw_enem_escola,Não existe
3,lnd_org_raw_saeb,lnd/crw/inep_saeb,org_raw_saeb,Não existe
4,lnd_org_raw_estrutura_territorial,lnd/crw/ibge,org_raw_estrutura_territorial,Não existe
5,lnd_org_raw_estrutura_cnae,lnd/crw/ibge,org_raw_estrutura_cnae,Não existe
6,lnd_org_raw_microdados_pnad,lnd/crw/ibge,org_raw_microdados_pnad,Não existe
7,lnd_org_raw_inse_nivel_socioeconomico,lnd/crw/inse,org_raw_inse_nivel_socioeconomico,Não existe
8,lnd_org_raw_ipca,lnd/crw/ibge,org_raw_ipca,Não existe
9,lnd_org_raw_estrutura_cbo,lnd/crw/ibge,org_raw_estrutura_cbo,Não existe


In [6]:
result_list

[{'NAME_PIPELINE': 'lnd_org_raw_censo_escolar',
  'FOLDER': 'lnd/crw/inep_censo_escolar',
  'BOT_SCRIPT_NAME': 'org_raw_censo_escolar',
  'MONITORAMENTO': 'Não existe'},
 {'NAME_PIPELINE': 'lnd_org_raw_microdados_enem',
  'FOLDER': 'lnd/crw/inep_enem',
  'BOT_SCRIPT_NAME': 'org_raw_microdados_enem',
  'MONITORAMENTO': 'Não existe'},
 {'NAME_PIPELINE': 'lnd_org_raw_enem_escola',
  'FOLDER': 'lnd/crw/inep_enem',
  'BOT_SCRIPT_NAME': 'org_raw_enem_escola',
  'MONITORAMENTO': 'Não existe'},
 {'NAME_PIPELINE': 'lnd_org_raw_saeb',
  'FOLDER': 'lnd/crw/inep_saeb',
  'BOT_SCRIPT_NAME': 'org_raw_saeb',
  'MONITORAMENTO': 'Não existe'},
 {'NAME_PIPELINE': 'lnd_org_raw_estrutura_territorial',
  'FOLDER': 'lnd/crw/ibge',
  'BOT_SCRIPT_NAME': 'org_raw_estrutura_territorial',
  'MONITORAMENTO': 'Não existe'},
 {'NAME_PIPELINE': 'lnd_org_raw_estrutura_cnae',
  'FOLDER': 'lnd/crw/ibge',
  'BOT_SCRIPT_NAME': 'org_raw_estrutura_cnae',
  'MONITORAMENTO': 'Não existe'},
 {'NAME_PIPELINE': 'lnd_org_raw_mic

In [3]:
pd.set_option('display.max_rows', None)

In [8]:
df

,NAME_PIPELINE,FOLDER,BOT_SCRIPT_NAME,MONITORAMENTO
0,lnd_org_raw_censo_escolar,lnd/crw/inep_censo_escolar,org_raw_censo_escolar,Não existe
1,lnd_org_raw_microdados_enem,lnd/crw/inep_enem,org_raw_microdados_enem,Não existe
2,lnd_org_raw_enem_escola,lnd/crw/inep_enem,org_raw_enem_escola,Não existe
3,lnd_org_raw_saeb,lnd/crw/inep_saeb,org_raw_saeb,Não existe
4,lnd_org_raw_estrutura_territorial,lnd/crw/ibge,org_raw_estrutura_territorial,Não existe
5,lnd_org_raw_estrutura_cnae,lnd/crw/ibge,org_raw_estrutura_cnae,Não existe
6,lnd_org_raw_microdados_pnad,lnd/crw/ibge,org_raw_microdados_pnad,Não existe
7,lnd_org_raw_inse_nivel_socioeconomico,lnd/crw/inse,org_raw_inse_nivel_socioeconomico,Não existe
8,lnd_org_raw_ipca,lnd/crw/ibge,org_raw_ipca,Não existe
9,lnd_org_raw_estrutura_cbo,lnd/crw/ibge,org_raw_estrutura_cbo,Não existe


In [34]:
import os
import re
import json

import pandas as pd


# Path to the folder

#json_file = r'C:\Users\PC\Desktop\python\mapeamento_adf\all_pipelines.json'
json_file = '././all_pipelines.json'
with open(json_file, 'r') as file:
    json_string = json.load(file)
    word = 'lnd'
    layer = 'lnd_org_raw'
    print(len(json_string))
    # List comprehension to filter data
    list_1 = [x for x in json_string if 'name' in x and x['name'].startswith(layer) and not x['name'].startswith('wkf')]
    print(len(list_1))
    name_pipeline = x.get('name', {})
    folder = x.get('properties', {}).get('folder', {}).get('name', {})
    bot_script = x.get('properties', {}).get('parameters', {}).get('bot', {}).get('defaultValue', {})
    #print(len([y for y in list_1]))
    for y in list_1:
        #print(y["properties"]['activities'])
        for z in y["properties"]['activities']:
            if z['name'] == 'alerta_falha':
                email_users = z['typeProperties']['parameters']['email_users']['value']
                print(email_users)
            else:
                pass

    #list_2 = [y["properties"]['activities'] for y in list_1]
    #for z in list_2:
    #    print(z)


2805
185
@array('monitoramento_observatorio@cni.com.br')
@array('monitoramento_observatorio@cni.com.br')
@array('monitoramento_observatorio@cni.com.br')
@array('monitoramento_observatorio@cni.com.br')
@array('monitoramento_observatorio@cni.com.br')
@array('monitoramento_observatorio@cni.com.br')
@array('monitoramento_observatorio@cni.com.br')
@array('monitoramento_observatorio@cni.com.br')
@array('monitoramento_observatorio@cni.com.br')
@array('monitoramento_observatorio@cni.com.br')
@array('monitoramento_observatorio@cni.com.br')
@array('monitoramento_observatorio@cni.com.br')
@array('monitoramento_observatorio@cni.com.br')
@array('monitoramento_observatorio@cni.com.br')
@array('monitoramento_observatorio@cni.com.br')
@array('monitoramento_observatorio@cni.com.br')
@array('monitoramento_observatorio@cni.com.br')
@array('monitoramento_observatorio@cni.com.br')
@array('monitoramento_observatorio@cni.com.br')
@array('monitoramento_observatorio@cni.com.br')
@array('monitoramento_observato

In [ ]:
import os
import json
import re
import pandas as pd

# Path to the folder

#json_file = r'C:\Users\PC\Desktop\python\mapeamento_adf\all_pipelines.json'
json_file = '././all_pipelines.json'
with open(json_file, 'r') as file:
    json_string = json.load(file)
    word = 'lnd'
    layer = 'lnd_org_raw'
#type(data) - list
    for x in json_string:
        if x['name'] and x['name'].startswith(layer) and not x['name'].startswith('wkf'): 
            #print(x["properties"]["activities"])
            if x["properties"]:
                #print(x['name'])
                #print(x['id'])
                #print(x['properties']['activities'])
                for y in x['properties']['activities']:
                    print(y['typeProperties']['parameters']['email_users'])
            else:
                pass

In [ ]:
import os
import json
import re
import pandas as pd

# Path to the folder

json_file = r'C:\Users\PC\Desktop\python\mapeamento_adf\all_pipelines.json'
with open(json_file, 'r') as file:
    json_string = json.load(file)
    word = 'lnd'
    layer = 'lnd_org_raw'
#type(data) - list
    for x in json_string:
        if x['name'] and x['name'].startswith(layer) and not x['name'].startswith('wkf'): 
            #print(x["properties"]["activities"])
            if x["properties"]:
                #print(x['name'])
                #print(x['id'])
                #print(x['properties']['activities'])
                for y in x['properties']['activities']:
                    print(y['typeProperties']['parameters']['email_users'])
            else:
                pass

In [114]:
import os
import json
import re
import pandas as pd

# Path to the folder

json_file = r'C:\Users\PC\Desktop\python\mapeamento_adf\all_pipelines.json'
with open(json_file, 'r') as file:
    json_string = json.load(file)
    word = 'lnd'
    layer = 'lnd_org_raw'
#type(data) - list
    for x in json_string:
        if x['name'] and x['name'].startswith(layer) and not x['name'].startswith('wkf'): 
             for y in x['properties']['activities']:
                if y['typeProperties']:

                    print(y['typeProperties']['pipeline']['referenceName'])
                else:
                    pass
        else:
            pass


import_lnd__0__execute_crw
import_lnd__0__execute_crw
import_lnd__0__execute_crw
import_lnd__0__execute_crw
import_lnd__0__execute_crw
import_lnd__0__execute_crw
import_lnd__0__execute_crw
import_lnd__0__execute_crw
import_lnd__0__execute_crw
import_lnd__0__execute_crw
import_lnd__0__execute_crw
import_lnd__0__execute_crw
import_lnd__0__execute_crw
import_lnd__0__execute_crw
import_lnd__0__execute_crw
import_lnd__0__execute_crw
import_lnd__0__execute_crw
import_lnd__0__execute_crw
send_email


KeyError: 'pipeline'

In [ ]:
{'id': '/subscriptions/3e3a21e3-9551-4945-8c46-c02f558392ce/resourceGroups/bigdata/providers/Microsoft.DataFactory/factories/cnibigdatafactory/pipelines/lnd_org_raw_inep_atu',
 'name': 'lnd_org_raw_inep_atu',
 'type': 'Microsoft.DataFactory/factories/pipelines', 
 'properties': {'activities': [{'name': 'import_lnd__0__execute_crw', 'type': 'ExecutePipeline', 'dependsOn': [], 'userProperties': [], 'typeProperties': 
      {'pipeline': {'referenceName': 'import_lnd__0__execute_crw', 'type': 'PipelineReference'}, 'waitOnCompletion': True, 'parameters': {'bot': {'value': '@pipeline().parameters.bot', 'type': 'Expression'}, 'adf': {'value': '@json(concat(\'{"adf_factory_name": "\', pipeline().DataFactory, \'", "adf_pipeline_name": "\', pipeline().Pipeline, \'", "adf_pipeline_run_id": "\', pipeline().RunId, \'",  "adf_trigger_id": "\', pipeline().TriggerId, \'", "adf_trigger_name": "\', pipeline().TriggerName, \'", "adf_trigger_time": "\', pipeline().TriggerTime, \'",  "adf_trigger_type": "\', pipeline().TriggerType, \'"}\'))', 'type': 'Expression'}, 'env': {'value': '@pipeline().parameters.env', 'type': 'Expression'}, 'bot_params': {'value': '@pipeline().parameters.bot_params', 'type': 'Expression'}}}}]
                    , 'concurrency': 1, 'policy': {'elapsedTimeMetric': {}, 'cancelAfter': {}}, 'parameters': {'bot': {'type': 'string', 'defaultValue': 'org_raw_inep_atu'}, 'env': {'type': 'object', 'defaultValue': {'env': 'dev'}}, 'bot_params': {'type': 'string', 'defaultValue': {'location': 'brazilsouth', 'reload': 'false', 'reset': 'false', 'override': 'false', 'memory_in_gb': '8', 'cpu': '4.0'}}}, 'folder': {'name': 'lnd/crw/inep_atu'}, 'annotations': ['landing', 'crawler'], 'lastPublishTime': '2023-04-19T17:13:39Z'}, 'etag': '7d00e23d-0000-0200-0000-644021430000'}

In [ ]:
[{'name': 'import_lnd__0__execute_crw', 'type': 'ExecutePipeline', 'dependsOn': [], 'userProperties': [], 'typeProperties': {'pipeline': {'referenceName': 'import_lnd__0__execute_crw', 'type': 'PipelineReference'}, 'waitOnCompletion': True, 'parameters': {'bot': {'value': '@pipeline().parameters.bot', 'type': 'Expression'}, 'adf': {'value': '@json(concat(\'{"adf_factory_name": "\', pipeline().DataFactory, \'", "adf_pipeline_name": "\', pipeline().Pipeline, \'", "adf_pipeline_run_id": "\', pipeline().RunId, \'",  "adf_trigger_id": "\', pipeline().TriggerId, \'", "adf_trigger_name": "\', pipeline().TriggerName, \'", "adf_trigger_time": "\', pipeline().TriggerTime, \'",  "adf_trigger_type": "\', pipeline().TriggerType, \'"}\'))', 'type': 'Expression'}, 'env': {'value': '@pipeline().parameters.env', 'type': 'Expression'}, 'bot_params': {'value': '@pipeline().parameters.bot_params', 'type': 'Expression'}}}}, {'name': 'alerta_falha', 'type': 'ExecutePipeline', 'dependsOn': [{'activity': 'import_lnd__0__execute_crw', 'dependencyConditions': ['Failed']}], 'userProperties': [], 'typeProperties': {'pipeline': {'referenceName': 'send_email', 'type': 'PipelineReference'}, 'waitOnCompletion': True, 'parameters': {'email_users': {'value': "@array('monitoramento_observatorio@cni.com.br')", 'type': 'Expression'}, 'email_groups': {'value': "@array('')", 'type': 'Expression'}, 'adf': {'value': '@json(concat(\'{"adf_factory_name": "\', pipeline().DataFactory, \'", "adf_pipeline_name": "\', pipeline().Pipeline, \'", "adf_pipeline_run_id": "\', pipeline().RunId, \'",  "adf_trigger_id": "\', pipeline().TriggerId, \'", "adf_trigger_name": "\', pipeline().TriggerName, \'", "adf_trigger_time": "\', pipeline().TriggerTime, \'",  "adf_trigger_type": "\', pipeline().TriggerType, \'"}\'))', 'type': 'Expression'}, 'env': {'value': '@pipeline().parameters.env', 'type': 'Expression'}}}}]

In [ ]:
{'id': '/subscriptions/3e3a21e3-9551-4945-8c46-c02f558392ce/resourceGroups/bigdata/providers/Microsoft.DataFactory/factories/cnibigdatafactory/pipelines/lnd_org_raw_rfb_cno', 'name': 'lnd_org_raw_rfb_cno', 'type': 'Microsoft.DataFactory/factories/pipelines', 'properties': {'activities': [{'name': 'import_lnd__0__execute_crw', 'type': 'ExecutePipeline', 'dependsOn': [], 'policy': {'secureInput': False}, 'userProperties': [], 'typeProperties': {'pipeline': {'referenceName': 'import_lnd__0__execute_crw', 'type': 'PipelineReference'}, 'waitOnCompletion': True, 'parameters': {'bot': {'value': '@pipeline().parameters.bot', 'type': 'Expression'}, 'adf': {'value': '@json(concat(\'{"adf_factory_name": "\', pipeline().DataFactory, \'", "adf_pipeline_name": "\', pipeline().Pipeline, \'", "adf_pipeline_run_id": "\', pipeline().RunId, \'",  "adf_trigger_id": "\', pipeline().TriggerId, \'", "adf_trigger_name": "\', pipeline().TriggerName, \'", "adf_trigger_time": "\', pipeline().TriggerTime, \'",  "adf_trigger_type": "\', pipeline().TriggerType, \'"}\'))', 'type': 'Expression'}, 'env': {'value': '@pipeline().parameters.env', 'type': 'Expression'}, 'bot_params': {'value': '@pipeline().parameters.bot_params', 'type': 'Expression'}}}}, {'name': 'alerta_falha', 'type': 'ExecutePipeline', 'dependsOn': [{'activity': 'import_lnd__0__execute_crw', 'dependencyConditions': ['Failed']}], 'policy': {'secureInput': False}, 'userProperties': [], 'typeProperties': {'pipeline': {'referenceName': 'send_email', 'type': 'PipelineReference'}, 'waitOnCompletion': True, 'parameters': {'email_users': {'value': "@array('monitoramento_observatorio@cni.com.br')", 'type': 'Expression'}, 'email_groups': {'value': "@array('')", 'type': 'Expression'}, 'adf': {'value': '@json(concat(\'{"adf_factory_name": "\', pipeline().DataFactory, \'", "adf_pipeline_name": "\', pipeline().Pipeline, \'", "adf_pipeline_run_id": "\', pipeline().RunId, \'",  "adf_trigger_id": "\', pipeline().TriggerId, \'", "adf_trigger_name": "\', pipeline().TriggerName, \'", "adf_trigger_time": "\', pipeline().TriggerTime, \'",  "adf_trigger_type": "\', pipeline().TriggerType, \'"}\'))', 'type': 'Expression'}, 'env': {'value': '@pipeline().parameters.env', 'type': 'Expression'}}}}, {'name': 'Falha', 'description': '500', 'type': 'Fail', 'dependsOn': [{'activity': 'alerta_falha', 'dependencyConditions': ['Succeeded']}], 'userProperties': [], 'typeProperties': {'message': 'Falha', 'errorCode': '500'}}, {'name': 'If Condition1', 'type': 'IfCondition', 'dependsOn': [{'activity': 'import_lnd__0__execute_crw', 'dependencyConditions': ['Completed']}], 'userProperties': [], 'typeProperties': {'expression': {'value': "@equals(activity('import_lnd__0__execute_crw').output.pipelineReturnValue.return.return.status_execution.exit, 300)", 'type': 'Expression'}, 'ifTrueActivities': [{'name': 'emit_execution_alert', 'type': 'ExecutePipeline', 'dependsOn': [], 'policy': {'secureInput': False}, 'userProperties': [], 'typeProperties': {'pipeline': {'referenceName': 'send_email', 'type': 'PipelineReference'}, 'waitOnCompletion': True, 'parameters': {'status': 'succeeded', 'email_users': {'value': "@array('monitoramento_observatorio@cni.com.br')", 'type': 'Expression'}, 'email_groups': {'value': "@array('')", 'type': 'Expression'}, 'adf': {'value': '@json(concat(\'{"adf_factory_name": "\', pipeline().DataFactory, \'", "adf_pipeline_name": "\', pipeline().Pipeline, \'", "adf_pipeline_run_id": "\', pipeline().RunId, \'",  "adf_trigger_id": "\', pipeline().TriggerId, \'", "adf_trigger_name": "\', pipeline().TriggerName, \'", "adf_trigger_time": "\', pipeline().TriggerTime, \'",  "adf_trigger_type": "\', pipeline().TriggerType, \'"}\'))', 'type': 'Expression'}, 'env': {'value': '@pipeline().parameters.env', 'type': 'Expression'}, 'message': 'no-new-file Pipeline executada com sucesso, porém não encontrou novos arquivos para download!'}}}]}}, {'name': 'return', 'type': 'SetVariable', 'dependsOn': [{'activity': 'If Condition1', 'dependencyConditions': ['Completed']}], 'policy': {'secureOutput': False, 'secureInput': False}, 'userProperties': [], 'typeProperties': {'variableName': 'pipelineReturnValue', 'value': [{'key': 'return', 'value': {'type': 'Expression', 'content': "@activity('import_lnd__0__execute_crw').output.pipelineReturnValue.return.return.status_execution.exit"}}], 'setSystemVariable': True}}], 'concurrency': 1, 'policy': {'elapsedTimeMetric': {}}, 'parameters': {'bot': {'type': 'string', 'defaultValue': 'org_raw_rfb_cno'}, 'env': {'type': 'object', 'defaultValue': {'env': 'dev'}}, 'bot_params': {'type': 'string', 'defaultValue': {'location': 'brazilsouth', 'reload': 'false', 'reset': 'false', 'override': 'false', 'memory_in_gb': '8', 'cpu': '4.0'}}}, 'folder': {'name': 'lnd/crw/rfb_cno__cadastro_nacional_de_obras'}, 'annotations': ['landing', 'crawler'], 'lastPublishTime': '2024-03-06T18:16:17Z'}, 'etag': '3400393d-0000-0200-0000-65e8b2f10000'}

In [92]:
testing =   {
        "id": "/subscriptions/3e3a21e3-9551-4945-8c46-c02f558392ce/resourceGroups/bigdata/providers/Microsoft.DataFactory/factories/cnibigdatafactory/pipelines/raw_trs_pintec_grau_obst_cnae_f",
        "name": "raw_trs_pintec_grau_obst_cnae_f",
        "type": "Microsoft.DataFactory/factories/pipelines",
        "properties": {
            "activities": [
                {
                    "name": "trusted__0__switch_env",
                    "description": "Calls trusted template that switches env.",
                    "type": "ExecutePipeline",
                    "dependsOn": [],
                    "userProperties": [],
                    "typeProperties": {
                        "pipeline": {
                            "referenceName": "trusted__0__switch_env",
                            "type": "PipelineReference"
                        },
                        "waitOnCompletion": true,
                        "parameters": {
                            "env": {
                                "value": "@pipeline().parameters.env",
                                "type": "Expression"
                            },
                            "tables": {
                                "value": "@pipeline().parameters.tables",
                                "type": "Expression"
                            },
                            "databricks": {
                                "value": "@pipeline().parameters.databricks",
                                "type": "Expression"
                            },
                            "adf": {
                                "value": "@json(concat('{\"adf_factory_name\": \"', pipeline().DataFactory, '\", \"adf_pipeline_name\": \"', pipeline().Pipeline, '\", \"adf_pipeline_run_id\": \"', pipeline().RunId, '\",  \"adf_trigger_id\": \"', pipeline().TriggerId, '\", \"adf_trigger_name\": \"', pipeline().TriggerName, '\", \"adf_trigger_time\": \"', pipeline().TriggerTime, '\",  \"adf_trigger_type\": \"', pipeline().TriggerType, '\"}'))",
                                "type": "Expression"
                            }
                        }
                    }
                },
                {
                    "name": "alerta_falha",
                    "type": "ExecutePipeline",
                    "dependsOn": [
                        {
                            "activity": "trusted__0__switch_env",
                            "dependencyConditions": [
                                "Failed"
                            ]
                        }
                    ],
                    "userProperties": [],
                    "typeProperties": {
                        "pipeline": {
                            "referenceName": "send_email",
                            "type": "PipelineReference"
                        },
                        "waitOnCompletion": true,
                        "parameters": {
                            "email_users": {
                                "value": "@array('monitoramento_observatorio@cni.com.br')",
                                "type": "Expression"
                            },
                            "email_groups": {
                                "value": "@array('')",
                                "type": "Expression"
                            },
                            "adf": {
                                "value": "@json(concat('{\"adf_factory_name\": \"', pipeline().DataFactory, '\", \"adf_pipeline_name\": \"', pipeline().Pipeline, '\", \"adf_pipeline_run_id\": \"', pipeline().RunId, '\",  \"adf_trigger_id\": \"', pipeline().TriggerId, '\", \"adf_trigger_name\": \"', pipeline().TriggerName, '\", \"adf_trigger_time\": \"', pipeline().TriggerTime, '\",  \"adf_trigger_type\": \"', pipeline().TriggerType, '\"}'))",
                                "type": "Expression"
                            },
                            "env": {
                                "value": "@pipeline().parameters.env",
                                "type": "Expression"
                            }
                        }
                    }
                }
            ],
            "policy": {
                "elapsedTimeMetric": {}
            },
            "parameters": {
                "tables": {
                    "type": "object",
                    "defaultValue": {
                        "schema": "ibge",
                        "table": "pintec_grau_obst_cnae",
                        "prm_path": "usr/ibge_pintec/FIEC_IBGE_pintec_grau_obst_cnae_mapeamento_unificado_trusted.xlsx",
                        "source": "crw/oni/ibge/pintec/grau_obst/grau_obst_cnae/",
                        "destination": "oni/ibge/pintec/grau_obst/grau_obst_cnae/"
                    }
                },
                "databricks": {
                    "type": "object",
                    "defaultValue": {
                        "trs": {
                            "notebook": "/trs/ibge/pintec/raw_trs_pintec_grau_obst_cnae_f"
                        }
                    }
                },
                "user_parameters": {
                    "type": "object",
                    "defaultValue": {
                        "null": "null"
                    }
                },
                "env": {
                    "type": "object",
                    "defaultValue": {
                        "env": "dev"
                    }
                }
            },
            "folder": {
                "name": "trs/ibge"
            },
            "annotations": [],
            "lastPublishTime": "2024-01-15T14:26:10Z"
        },
        "etag": "4b011aed-0000-0200-0000-65a540820000"
    }

NameError: name 'true' is not defined

In [34]:
import os
import json
import re
import pandas as pd

# Path to the folder

json_file = r'C:\Users\PC\Desktop\python\mapeamento_adf\all_pipelines.json'
with open(json_file, 'r') as file:
    json_string = json.load(file)
    word = 'lnd'
    layer = 'lnd_org_raw'
#type(data) - list
    for x in json_string:
        if x['name'] and x['name'].startswith(layer) and not x['name'].startswith('wkf'): 
            name_pipeline = x.get('name', {})
            folder = x.get('properties', {}).get('folder', {}).get('name', {})
            bot_script = x.get('properties', {}).get('parameters', {}).get('bot', {}).get('defaultValue', {})


            dados = {
                 'NAME_PIPELINE': name_pipeline,
                 'FOLDER': folder,
                 'BOT_SCRIPT_NAME': bot_script}
            
            result_list.append(dados)

# Assuming result_list is your list of dictionaries
df = pd.DataFrame(result_list)

In [35]:
df

,NAME_PIPELINE,FOLDER,BOT_SCRIPT_NAME
0,lnd_org_raw_aco_brasil_prod_aco_nacional,lnd/crw/aco_brasil,org_raw_aco_brasil_prod_aco_nacional
1,lnd_org_raw_aneel_gera_energia_distrib,lnd/crw/aneel,org_raw_gera_energia_distribuida
2,lnd_org_raw_aneel_gera_energia_distrib_adb,lnd_adb,org_raw_gera_energia_distribuida
3,lnd_org_raw_bacen_cotacao,lnd/crw/oni/bacen,org_raw_bacen_cotacao
4,lnd_org_raw_bacen_estban_mensal_municipios,lnd/crw/oni/bacen/estban_mensal,org_raw_bacen_estban_mensal_municipios
...,...,...,...
541,lnd_org_raw_mcti_empresas_lei_do_bem,lnd/crw/oni/mcti,org_raw_mcti_empresas_lei_do_bem
542,lnd_org_raw_bndes_operacoes_indiretas_automaticas,lnd/crw/oni/bndes,org_raw_bndes_operacoes_indiretas_automaticas
543,lnd_org_raw_cnpq_financiamento,lnd/crw/oni/cnpq,org_lnd_raw_cnpq_financiamento
544,lnd_org_raw_bndes_operacoes_nao_automaticas,lnd/crw/oni/bndes,org_raw_bndes_operacoes_nao_automaticas


# org_raw

In [37]:
import os
import json
import re
import pandas as pd

# Path to the folder

json_file = r'C:\Users\PC\Desktop\python\mapeamento_adf\all_pipelines.json'
with open(json_file, 'r') as file:
    json_string = json.load(file)
    word = 'org'
    layer = 'org_raw'
#type(data) - list
    for x in json_string:
        if x['name'] and x['name'].startswith(layer) and not x['name'].startswith('wkf'):
            print(x['name'])


org_raw_inddesempenho_vw_lanc_evento_vl_r_ssi
org_raw_inddesempenho_empresa_dn
org_raw_prm_cod_detalhamento_negocio_x_cr
org_raw_base_escolas
org_raw_saeb_aluno
org_raw_saeb_diretor
org_raw_escola
org_raw_curso_educacao_profissional
org_raw_docente
org_raw_etapa_ensino
org_raw_enem_escola_dicionario
org_raw_gestor
org_raw_prova_brasil_aluno
org_raw_matriculas
org_raw_saeb_escola
org_raw_saeb_professor
org_raw_turmas
org_raw_estrutura_territorial
org_raw_enem_escola
org_raw_robo_legisdata_nivel_classificacao
org_raw_robo_legisdata_comissao
org_raw_robo_legisdata_tipo_casa_origem
org_raw_robo_legisdata_proposicao_legislativa
org_raw_robo_legisdata_resultado_proposicao
org_raw_robo_legisdata_posicao
org_raw_robo_legisdata_complemento_forma
org_raw_robo_legisdata_classificacao_proposicao
org_raw_robo_legisdata_distribuicao_comissao
org_raw_robo_legisdata_escala_prioridade
org_raw_robo_legisdata_autor
org_raw_robo_legisdata_classificacao
org_raw_robo_legisdata_divisao
org_raw_robo_legisdata

In [ ]:
import os
import json
import re

# Path to the folder
folder_path = r'C:\Users\PC\Desktop\mapeamento_adf\pipeline\pipeline'
# List all files in the folder
files = os.listdir(folder_path)
# ['biz_biz_analitico_ep.json', 'biz_biz_base_unimercado_ep.json', 'biz_biz_base_unimercado_ssi.json', 'biz_biz_base_unimercado_sti.json', ...
result_list = [] 
for file_name in files:
    # biz_biz_analitico_ep.json
    file_path = os.path.join(folder_path, file_name)
    # 'C:\Users\PC\Desktop\mapeamento_adf\pipeline\pipeline' + ['biz_biz_analitico_ep.json', 'biz_biz_base_unimercado_ep.json', 'biz_biz_base_unimercado_ssi.json', 'biz_biz_base_unimercado_sti.json', ...
   
    ## with open(file_path, 'r') as file:
        ## data = json.load(file)
        # Now 'data' contains the JSON data as a Python dictionary
        # Convert the JSON object to a text string
        ## json_string = json.dumps(data, indent=2)
    json_string = funcao_carregar_e_conversao_json_file(file_path)    
    word = 'org'
    layer = 'org_raw'
    if word in json_string:
        # if re.search(rf'\b{word}\b', json_string, re.IGNORECASE) and not file_name.startswith('wkf') and file_name.startswith(layer):
        if word in json_string and file_name.startswith(layer) and not file_name.startswith('wkf'): 
            #>> it doesn't work, cause the expression 'if word in json_string' ain't looking for the 'word in the specific sequence'
            #print(file_name)
            with open(file_path, 'r') as file:
                data = json.load(file)

                name_pipeline = data.get('name', {})
                folder = data.get('properties', {}).get('folder', {}).get('name', {})

                #print(data)
                #print('NAME_PIPELINE', name_pipeline)
                #print('FOLDER', folder)  
                text = data.get('properties', {}).get('parameters', {}).get('files', {}).get('defaultValue', {})
                #print(text)
                if data.get('properties', {}).get('parameters', {}).get('files', {}).get('defaultValue', {}) != {} and re.search(r'\bfiesc\b', str(text), re.IGNORECASE):
                    #print(data.get('properties', {}).get('parameters', {}).get('files', {}).get('defaultValue', {})[0])
                    #example = data.get('properties', {}).get('parameters', {}).get('files', {}).get('defaultValue', {})[0].replace("{", "")

                    #data = json.loads(example, indent=4)
                    #print(data)
                    ## print(data)
                    ## print('FIESC')

                    print('1_PATH')
                    print('NAME_PIPELINE', name_pipeline)
                    print('FOLDER', folder)
                    #print('PARAMETERS', filtered_list)  
                

                    #print(type(data.get('properties', {}).get('parameters', {}).get('files', {}).get('defaultValue', {})[0]))
                elif data.get('properties', {}).get('parameters', {}).get('files', {}).get('defaultValue', {}) != {}:
                    data_get = data.get('properties', {}).get('parameters', {}).get('files', {}).get('defaultValue', {})
                    # Convert the single-quoted string to a valid JSON object by replacing single quotes with double quotes
                    json_str = data_get[0].replace("'", '"')
                    # Parse the string into a JSON object (dictionary)
                    json_obj = json.loads(json_str)

                    print('2_PATH')
                    print('NAME_PIPELINE', name_pipeline)
                    print('FOLDER', folder)  
                    #print(json_obj.values())
                    filtered_list = [item for item in json_obj.values()]
                    print('PARAMETERS', filtered_list)

                    #json_ob.values()
                    # Write the JSON object to a file
                    #with open('output.json', 'w', encoding='utf-8') as json_file:
                    #    json.dump(json_obj, json_file, ensure_ascii=False, indent=4)
                    #    print(json.dump(json_obj, json_file, ensure_ascii=False, indent=4))

                    

                else:
                    print('3_PATH')
                    print('NAME_PIPELINE', name_pipeline)
                    print('FOLDER', folder)  
                    data_get = data.get('properties', {}).get('parameters', {}).get('tables', {}).get('defaultValue', {})
                    print('PARAMETERS', data_get)

                    #print(file_name)

                ### # Check if the key 'Values' exists in the dictionary
                ### data_properties_parameteres = data.get('properties', {}).get('parameters', {})
                ### if "databricks" in data_properties_parameteres:
                ###     print('databricks')
                    # Assign the value of 'Values' to the new key 'NewValues'
                    #data_properties_parameteres["tables"] = data_properties_parameteres.pop("databricks")
                    #data_properties_parameteres = {"tables": data_properties_parameteres["tables"], **data_properties_parameteres}
                    #databricks_local = data_properties_parameteres.get('tables', {}).get('defaultValue', {}).get('notebook', {})

                ### else:
                ###     print('NAO TEM databricks')
                ###     print(file_path)
                    #print(data_properties_parameteres)

                    #["{ 'namespace': '/oni', 
                    # 'file_folder': '/pesquisas/scg/absenteismo', 
                    # 'file_folder_raw':'/oni/pesquisas/scg/absenteismo', 
                    # 'extension': 'csv', 'column_delimiter': ';', 
                    # 'encoding': 'UTF-8', 'null_value': '' }"]

In [2]:

import pandas as pd

# Assuming result_list is your list of dictionaries
df = pd.DataFrame(result_list)

# Save the DataFrame to a CSV file
df.to_csv('lnd_org_raw.csv', index=False)

NameError: name 'result_list' is not defined

In [52]:
import csv

# Corrected code to open and read the CSV file
with open(r'C:\Users\PC\Desktop\mapeamento_adf\lnd_org_raw.csv', 'r') as file:
    reader = csv.reader(file)
    
    # Example: Iterate through the rows and print them
    for row in reader:
        print(row)


['NAME_PIPELINE', 'FOLDER', 'BOT_SCRIPT_NAME']
['lnd_org_raw_aco_brasil_prod_aco_nacional', 'lnd/crw/aco_brasil', 'org_raw_aco_brasil_prod_aco_nacional']
['lnd_org_raw_aneel_gera_energia_distrib', 'lnd/crw/aneel', 'org_raw_gera_energia_distribuida']
['lnd_org_raw_aneel_gera_energia_distrib_adb', 'lnd_adb', 'org_raw_gera_energia_distribuida']
['lnd_org_raw_bacen_cotacao', 'lnd/crw/oni/bacen', 'org_raw_bacen_cotacao']
['lnd_org_raw_bacen_estban_mensal_municipios', 'lnd/crw/oni/bacen/estban_mensal', 'org_raw_bacen_estban_mensal_municipios']
['lnd_org_raw_banco_mundial_indicadores_selecionados', 'lnd/crw/banco_mundial', 'org_raw_banco_mundial_indicadores_selecionados']
['lnd_org_raw_banco_mundial_preco_commodity', 'lnd/crw/banco_mundial', 'org_raw_banco_mundial_preco_commodity']
['lnd_org_raw_camara_dep_dados_abertos_atividade_parlamentar', 'lnd/crw/oni/camara_dep/dados_abertos', 'org_raw_camara_dep_dados_abertos_atividade_parlamentar']
['lnd_org_raw_camara_dep_dados_abertos_deputados', 'l

In [ ]:
help = 'Load data from CSV file into the database'


csv_file = kwargs[r'C:\Users\PC\Desktop\django_api_v1\mysite\api\management\commands\lnd_org_raw.csv']
with open(csv_file, 'r') as file:
    reader = csv.reader(file)
    next(reader)  # Skip header row
    for row in reader:
        Data.objects.create(
            column1=row[0],
            column2=row[1],
            column3=row[2],
            # Map fields to your model fields
        )
self.stdout.write(self.style.SUCCESS('Data loaded successfully'))


In [ ]:
[{'name': 'import_lnd__0__execute_crw', 'type': 'ExecutePipeline', 'dependsOn': [], 'userProperties': [], 'typeProperties': {'pipeline': {'referenceName': 'import_lnd__0__execute_crw', 'type': 'PipelineReference'}, 'waitOnCompletion': True, 'parameters': {'bot': {'value': '@pipeline().parameters.bot', 'type': 'Expression'}, 'adf': {'value': '@json(concat(\'{"adf_factory_name": "\', pipeline().DataFactory, \'", "adf_pipeline_name": "\', pipeline().Pipeline, \'", "adf_pipeline_run_id": "\', pipeline().RunId, \'",  "adf_trigger_id": "\', pipeline().TriggerId, \'", "adf_trigger_name": "\', pipeline().TriggerName, \'", "adf_trigger_time": "\', pipeline().TriggerTime, \'",  "adf_trigger_type": "\', pipeline().TriggerType, \'"}\'))', 'type': 'Expression'}, 'env': {'value': '@pipeline().parameters.env', 'type': 'Expression'}, 'bot_params': {'value': '@pipeline().parameters.bot_params', 'type': 'Expression'}}}}]

In [ ]:
import os
import json
import re
# Path to the folder

folder_path = r'C:\Users\PC\Desktop\mapeamento_adf\pipeline\pipeline'

# List all files in the folder
files = os.listdir(folder_path)
# ['biz_biz_analitico_ep.json', 'biz_biz_base_unimercado_ep.json', 'biz_biz_base_unimercado_ssi.json', 'biz_biz_base_unimercado_sti.json', ...

for file_name in files:
    # biz_biz_analitico_ep.json
    # biz_biz_base_unimercado_ep.json
    # biz_biz_base_unimercado_ssi.json
    # biz_biz_base_unimercado_sti.json
    # biz_biz_calcula_coluna.json


    file_path = os.path.join(folder_path, file_name)
    # 'C:\Users\PC\Desktop\mapeamento_adf\pipeline\pipeline' + ['biz_biz_analitico_ep.json', 'biz_biz_base_unimercado_ep.json', 'biz_biz_base_unimercado_ssi.json', 'biz_biz_base_unimercado_sti.json', ...
    with open(file_path, 'r') as file:
        data = json.load(file)

        # Now 'data' contains the JSON data as a Python dictionary
        # Convert the JSON object to a text string
        json_string = json.dumps(data, indent=2)
        word = 'rais'
        layer = 'trs_biz'
        if word in json_string:
            # if re.search(rf'\b{word}\b', json_string, re.IGNORECASE) and not file_name.startswith('wkf') and file_name.startswith(layer):
            if word in json_string and file_name.startswith('trs_biz') and not file_name.startswith('wkf'): 
                #>> it doesn't work, cause the expression 'if word in json_string' ain't looking for the 'word in the specific sequence'
                #print(file_name)
                with open(file_path, 'r') as file:
                    data = json.load(file)
                    print(data)
                    path_destination = data.get('properties', {}).get('parameters', {}).get('tables', {}).get('defaultValue', {}).get('path_destination')
                    # dictionary.get(key, default=None)
                    #print(data.get('properties', {}).get('parameters', {}))
                    if path_destination:
                        # if every single one exists data.get('properties', {}).get('parameters', {}).get('tables', {}).get('defaultValue', {}).get('path_destination')
                        print('path_destination' , data['properties']['parameters']['tables']['defaultValue']['path_destination'])
                        print('defaultValue' , data['properties']['parameters']['tables']['defaultValue'])
                        data_dict_values = data['properties']['parameters']['tables']['defaultValue']    
                        #for value in data_dict_values.values():
                        #    print('VALUES',value)
                        #valor = [value for value in data_dict_values.values()]
                        #print(valor)
                        #print(path_destination)
                        #if path_destination not in valor:
                        #    print('sdfsdfsf')

                        # Use a list comprehension with a condition to filter items
                        filtered_list = [item for item in data_dict_values.values() if not re.search(rf'\b{path_destination}\b', str(item), re.IGNORECASE)]
                        print("Filtered list excluding specific word:", filtered_list[:-1])
                    else:
                        path_destination = data.get('properties', {}).get('parameters', {}).get('tables', {}).get('defaultValue', {}).get('destination')
                        data_dict_values = data['properties']['parameters']['tables']['defaultValue']
                        #for value in data_dict_values.values():
                        #    print('VALUES',value)
                        #valor = [value for value in data_dict_values.values()]
                        #print(valor)
                        #print(path_destination)
                        #if path_destination not in valor:
                        #    print('sdfsdfsf')

                        # Use a list comprehension with a condition to filter items
                        filtered_list = [item for item in data_dict_values.values() if not re.search(rf'\b{path_destination}\b', str(item), re.IGNORECASE)]
                        print("Filtered list excluding specific wordFiltered list excluding specific wordFiltered list excluding specific word:", filtered_list[:-1])

        else:
            pass